In [4]:
# //
# //                       _oo0oo_
# //                      o8888888o
# //                      88" . "88
# //                      (| -_- |)
# //                      0\  =  /0
# //                    ___/`---'\___
# //                  .' \\|     |// '.
# //                 / \\|||  :  |||// \
# //                / _||||| -:- |||||- \
# //               |   | \\\  -  /// |   |
# //               | \_|  ''\---/''  |_/ |
# //               \  .-\__  '-'  ___/-. /
# //             ___'. .'  /--.--\  `. .'___
# //          ."" '<  `.___\_<|>_/___.' >' "".
# //         | | :  `- \`.;`\ _ /`;.`/ - ` : | |
# //         \  \ `_.   \_ __\ /__ _/   .-` /  /
# //     =====`-.____`.___ \_____/___.-`___.-'=====
# //                       `=---='
# //
# //
# //     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# //
# //               佛祖保佑         永无BUG
# //
# //
# //

In [13]:
# import requests
# from lxml import etree
# import time

# class Lagou_Job_Deeplearning(object):
#     def __init__(self):
#         self.base_url = 'https://www.lagou.com/zhaopin/shenduxuexi/'
#         self.headers = {
#             'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
#         }
#         self.job_web = []
        
#     def get_response(self, url):
#         html = requests.get(url, headers=self.headers).content.decode('utf-8')
#         return html
    
#     def prase_data(self, html):
#         xpath_data = etree.HTML(html)
#         web_list = xpath_data.xpath('//*[@class="p_top"]/a[@class="position_link"]/@href')
#         for web in web_list:
#             self.job_web.append(web)
            
#         return self.job_web

    
#     def start(self):
#         count = 0
#         for i in range(1,3):
#             count += 15
#             print('已爬取%s个'%count)
#             url = self.base_url + str(i)
#             response_html = self.get_response(url)
#             data = self.prase_data(response_html)
#             time.sleep(2)
#             print(data)

#         print('共%s个职位'%len(data))
#         return data
          
        

In [14]:
# job_web_list = Lagou_Job_Deeplearning().start()

In [63]:
from selenium import webdriver

browser = webdriver.Chrome('/Users/xufan/Desktop/chromedriver')
browser.delete_all_cookies()
browser.get('https://www.lagou.com/jobs/5826843.html')

username = '17600991883'
password = 'a6676610'

browser.find_element_by_xpath('//*[@id="lg_tbar"]/div/ul/li[1]/a').click()
browser.find_element_by_xpath('/html/body/section/div[2]/div[1]/div[2]/form/div[1]/input').send_keys(username)
browser.find_element_by_xpath('/html/body/section/div[2]/div[1]/div[2]/form/div[2]/input').send_keys(password)

time.sleep(3)
browser.find_element_by_xpath('/html/body/section/div[2]/div[1]/div[2]/form/div[5]/input').click()
time.sleep(10)

cookie_list = browser.get_cookies()
browser.get('https://www.lagou.com/zhaopin/shenduxuexi/?labelWords=label')
browser.delete_all_cookies()
for cookie in cookie_list:
    browser.add_cookie(cookie)

In [64]:
job_web_list = []
base_url = 'https://www.lagou.com/zhaopin/shenduxuexi/'
count = 0

for i in range(1,30):
    if i%5 == 0:
        print('已爬取%s页'%i)
    url = base_url + str(i)
    browser.get(url)
    html = browser.page_source
#     time.sleep(1)
    xpath_data = etree.HTML(html)
    web_list = xpath_data.xpath('//*[@class="p_top"]/a[@class="position_link"]/@href')
    for web in web_list:
        job_web_list.append(web)
print(len(job_web_list))

已爬取5页
已爬取10页
已爬取15页
已爬取20页
已爬取25页
435


In [69]:
browser = webdriver.Chrome('/Users/xufan/Desktop/chromedriver')
browser.get('https://www.lagou.com/')
browser.delete_all_cookies()
for cookie in cookie_list:
    browser.add_cookie(cookie)
htmls = []
p = 0
for url in job_web_list[:]:
    p += 1
    if p % 50 == 0:
        print('已爬取%s个'%p)
#     time.sleep(6)
    browser.get(url)
    html = browser.page_source
    xpath_data = etree.HTML(html)
#     if xpath_data.xpath('/html/body/div/p[1]/text()')[0] == '当前账号存在异常，请验证后继续访问！':
    if '当前账号存在异常' in html:
        time.sleep(10)
        browser.get(url)
        html = browser.page_source
    htmls.append(html)
print('共%s个职位'%len(htmls))

已爬取50个
已爬取100个
已爬取150个
已爬取200个
已爬取250个
已爬取300个
已爬取350个
已爬取400个
共435个职位


In [71]:
f = open('职位.txt','w')
for i in range(len(htmls)):
    xpath_data = etree.HTML(htmls[i])
#     job = xpath_data.xpath('/html/body/div[2]/div/div[1]/div/@title')
#     keywords = xpath_data.xpath('/html/body/div[2]/div/div[1]/dd/p[1]/span/text()')
    describes = xpath_data.xpath('//*[@class="job-detail"]/p/text()')
    
#     f.write(','.join(job))
#     f.write(','.join(keywords))
#     f.write('\n')
    f.write(','.join(describes))
    f.write('\n')

print('finish')
f.close()

finish


In [88]:
import pandas as pd
import numpy as np
import csv

job_dataframe_list = []
for i in range(len(htmls)):
    xpath_data = etree.HTML(htmls[i])
    
    company = xpath_data.xpath('//*[@id="job_company"]/dt/a/div/h2/em/text()')
    job = xpath_data.xpath('/html/body/div[2]/div/div[1]/div/@title')
    salary = xpath_data.xpath('/html/body/div[2]/div/div[1]/dd/p[1]/span[1]/text()')
    place = xpath_data.xpath('/html/body/div[2]/div/div[1]/dd/p[1]/span[2]/text()')
    experience = xpath_data.xpath('/html/body/div[2]/div/div[1]/dd/p[1]/span[3]/text()')
    education = xpath_data.xpath('/html/body/div[2]/div/div[1]/dd/p[1]/span[4]/text()')
    describes = xpath_data.xpath('//*[@class="job-detail"]/p/text()')
    
    temp = []
    temp.append(','.join(company))
    temp.append(','.join(job))
    temp.append(','.join(salary))
    temp.append(','.join(education))
    temp.append(','.join(place))
    temp.append(','.join(experience))
    temp.append(','.join(describes))
    job_dataframe_list.append(temp)
    
dataframe = pd.DataFrame(job_dataframe_list, 
                         columns=['company','job','salary','education','place','experience','describes'])
                         
dataframe.to_csv('职位.csv',index=0)

## 查看原始数据数据

In [89]:
data = pd.read_csv('职位.csv')
print(data.shape)
data.sample(5)

(435, 7)


,company,job,salary,education,place,experience,describes
130,\n 快手\n ...,分布式深度学习平台开发工程师,30k-50k,本科及以上 /,/北京 /,经验3-5年 /,"1、参与大规模深度学习平台的功能设计和研发；,2、提升线上计算和线下训练的GPU利用率。,1..."
56,\n 悉见科技\n ...,深度学习主任科学家,40k-80k,博士及以上 /,/北京 /,经验5-10年 /,"岗位职责：,1. 搭建视觉机器学习平台；,2. 根据公司技术战略，进行前沿的深度学习视觉算法..."
320,\n 用友烟草软件\n...,erp实施顾问,6k-8k,本科及以上 /,/北京 /,经验1-3年 /,"工作职责： 负责公司产品的实施工作，参与BU的项目运维工作。,职位要求：,1.全日制本科及以..."
167,\n 朗播\n ...,学习规划师,4k-8k,本科及以上 /,/北京 /,经验不限 /,"朗播英语智能学习实验室简介,朗播 | 英语智能学习实验室（朗播线下的教育新零售业务），是朗播..."
13,\n HoloMati...,深度学习（视觉）工程师,15k-30k,学历不限 /,/北京 /,经验1-3年 /,"岗位职责：,负责高精度导航地图中的各类地图元素的检测，识别和定位，主要包括但不限于以下几项:..."


## 清洗数据
### 格式

In [84]:
# 删除空值行
data.dropna(axis=0, how='any', inplace=True)
data.shape

(409, 6)

In [90]:
import re


for i in range(data.shape[0]):
    
    try:
        # 计算平均工资
        find_salary = re.findall(r'\d*\d',data['salary'][i])
        avg_salary = int((int(find_salary[0])+int(find_salary[1]))/2*1000)
        data['salary'][i] = avg_salary
        # 去掉斜杠
        data['company'][i] = re.sub(r'[\n]',' ',data['company'][i])
        data['education'][i] = re.sub(r'[/]',' ',data['education'][i])
        data['place'][i] = re.sub(r'[/]',' ',data['place'][i])
        data['experience'][i] = re.sub(r'[/]',' ',data['experience'][i])
    except:
        pass

data.head(5)

,company,job,salary,education,place,experience,describes
0,Udesk－企业级...,机器学习,22500,硕士及以上,北京,经验1-3年,"职位描述：,+ 企业服务领域智能化场景的业务分析、数据建模和数据优化。,任职要求：,+ 1年..."
1,佰策科技 ...,深度学习工程师,15000,本科及以上,北京,经验3-5年,深度学习工程师是最神秘的职业，在我们这里也不例外，不同的是，深度学习工程师是我们这里最能驱动...
2,小狗机器人 ...,深度学习算法研究员,37500,本科及以上,北京,经验3-5年,"岗位职位：,1、跟踪最前沿的视觉算法，对论文结果进行复现和效果提升。,2、针对具体场景和平台..."
3,厚仁医疗 ...,深度学习训练系统,50000,本科及以上,北京,经验3-5年,负责深度学习训练系统平台的效率优化和功能开发，支撑语音、NLP业务的模型训练。
4,印迹 ...,深度学习算法工程师,37500,硕士及以上,北京,经验3-5年,NaN


## 提取关键词并设置特征

In [91]:
import jieba.analyse

# 读入乱乱的txt文件
with open('职位.txt','r') as f:
    txt = f.readlines()
    txt = ','.join(txt)

# 去除标点符号等
def mysplit(file):
    '''
    return a little clean list
    '''
    with open(file,'r') as f:
        l = []
        for i in f:
            l.extend(re.split(r"\s'|'\s|^'|'$|[\[\]:;\s\d,.!?()\"/、。，：；-]",i))
    L = [i for i in l if i != '']
    return L

def key_words_list(num=15):
    cleaner_list = mysplit('职位.txt')
    txt = ','.join(cleaner_list)
    a = jieba.analyse.extract_tags(txt, topK=num, withWeight=False, allowPOS=())
    return a

feature = key_words_list(20)
print(feature)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/tc/mql7lrgs1vl2wf2j77td1zp00000gn/T/jieba.cache
Loading model cost 0.891 seconds.
Prefix dict has been built succesfully.


['学习', '算法', '深度', '经验', '优先', '能力', '熟悉', '相关', '机器', '工作', '负责', '岗位职责', '产品', '以上学历', '团队', '开发', '任职', '客户', '沟通', '数据']


In [92]:
# 建立简易词库
userdict = 'TensorFlow,机器学习,数据挖掘,AI,算法,深度学习,硕士,计算机,CNN,RNN,LSTM,Caffe,MXNet,编程,数学,python,强化学习,图像处理,Kaggle,github,全职,实习,985,NLP,SVM,LR,大规模,分布式,自然语言处理,沟通'
with open('userdict.txt','w') as f:
    for i in userdict.split(','):
        f.write(i)
        f.write('\n')

jieba.load_userdict("userdict.txt")

with open('userdict.txt','w') as f:
    for i in userdict:
        f.write(i)
        f.write('\n')

# 设置关键词为特征
for i in feature:
    a = np.array(np.zeros(435)).reshape(435,1)
    for j in range(435):
        # 读取词库并分词
        try:
            jieba.load_userdict("userdict.txt")
            words = jieba.cut(data['describes'][j])
            words = ','.join(words).split(',')
        except:
            pass
        
        if i in words:
            a[j] = 1
        else:
            a[j] = 0
        
    data[i] = a

## 创建训练集

In [96]:
raw_data = data.iloc[:,7:]
raw_data.sample(5)

,学习,算法,深度,经验,优先,能力,熟悉,相关,机器,工作,负责,岗位职责,产品,以上学历,团队,开发,任职,客户,沟通,数据
200,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
197,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
284,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [97]:
raw_label = data.iloc[:,2:3]
raw_label.head()

,salary
0,22500
1,15000
2,37500
3,50000
4,37500


In [98]:
raw_label.median()

salary    22500.0
dtype: float64

可以看出收入薪资的中位数为24000，所以设置大于24000的为1，小于24000的为0

In [99]:
# 设置大于24000的为1，小于24000的为0
raw_label['salary'] = raw_label.salary.apply(lambda x: 0 if x <= 24000 else 1)
raw_label.head()

,salary
0,0
1,0
2,1
3,1
4,1


In [100]:
print(raw_data.shape,raw_label.shape)

(435, 20) (435, 1)


In [104]:
# 分出训练集和测试集
train_data = raw_data.iloc[:300,:]
test_data = raw_data.iloc[300:,:]
train_label = raw_label[:300]
test_label = raw_label[300:]
print(train_data.shape,train_label.shape)
print(test_data.shape,test_label.shape)

(300, 20) (300, 1)
(135, 20) (135, 1)


## 构造神经网络模型

In [105]:
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils

train_label = np_utils.to_categorical(train_label, 2)
test_label = np_utils.to_categorical(test_label, 2)


model = Sequential()
model.add(Dense(input_dim=20, units=1000))
model.add(Activation('relu'))
model.add(Dense(units=1000))
model.add(Activation('relu'))
model.add(Dense(units=1000))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_data, train_label,
          epochs=20,
          batch_size=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
300/300 [==============================] - 1s 2ms/step - loss: 0.6591 - acc: 0.5967
Epoch 2/20
300/300 [==============================] - 0s 408us/step - loss: 0.5796 - acc: 0.7133
Epoch 3/20
300/300 [==============================] - 0s 440us/step - loss: 0.5610 - acc: 0.6900
Epoch 4/20
300/300 [==============================] - 0s 386us/step - loss: 0.5649 - acc: 0.6567
Epoch 5/20
300/300 [==============================] - 0s 410us/step - loss: 0.5055 - acc: 0.7300
Epoch 6/20
300/300 [==============================] - 0s 394us/step - loss: 0.4665 - acc: 0.7533
Epoch 7/20
300/300 [==============================] - 0s 451us/step - loss: 0.4399 - acc: 0.7900
Epoch 8/20
300/300 [==============================] - 0s 404us/step - loss: 0.3923 - acc: 0.8167
Epoch 9/20
300/300 [==============================] - 0s 381us/step - loss: 0.3471 - acc: 0.8633
Epoch 10/20
300/300 [==============================] - 0s 425us/step - loss: 0.30

In [106]:
score = model.evaluate(test_data, test_label, batch_size=70)
print('Total loss in training set:', score[0])
print('Accuracy in training set:', score[1])

135/135 [==============================] - 0s 516us/step
Total loss in training set: 1.6052526765399509
Accuracy in training set: 0.5703703805252358
